<a href="https://colab.research.google.com/github/joydeep03564/Machine_Learning-/blob/master/Semantic_Image_Seg_Polyp_Det.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importing the necessary libraries 

import numpy as np
from sklearn.model_selection import train_test_split
import cv2
import os  # This library is used to join the paths of the image folder
from glob import glob
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model


In [ ]:
# Load dataset # Data sanity check: splited the data set into training,validation and test
def load_data(path, split=0.1):
    images = sorted(glob(os.path.join(path, "images/*")))
    masks = sorted(glob(os.path.join(path, "masks/*")))

    total_size=len(images)
    valid_size=int(split*total_size)
    test_size=int(split*total_size)
    # Spliting the whole dataset into training and validation according to the split values
    train_x,valid_x=train_test_split(images,test_size=valid_size,random_state=42)
    train_y,valid_y=train_test_split(masks,test_size=valid_size,random_state=42)
    # Spliting again the training set into test using the 'test size' split 
    train_x,test_x=train_test_split(train_x,test_size=test_size,random_state=4)
    train_y,test_y=train_test_split(train_y,test_size=test_size,random_state=4)
    
    

    return (train_x,train_y), (valid_x,valid_y),(test_x,test_y)



In [ ]:
def read_image(path):
    path=path.decode()
    x = cv2.imread(path,cv2.IMREAD_COLOR)
    x = cv2.resize(x,(256,256))
    x=  x/255.0
    #x=  x.astype(np.float32) #check this line
    return x

In [ ]:
def read_mask(path):
    path=path.decode()
    x=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x,(256,256))
    x=np.expand_dims(x,axis=-1)
    x=  x/255.0
    #x=  x.astype(np.float32) # check this line
    return x

In [ ]:
# tf_parse is nothing but reading the images and and then converting it to the tensorflow type.
# It is kind of pre-processed image which is going to used by tf.data.Dataset while creating the datasource for the pipeline
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
    x.set_shape([256, 256, 3])
    y.set_shape([256, 256, 1])
    return x, y
#This tf_dataset function is the main function. 
#To create an input pipeline, you must start with a data source. 
#For example, to construct a Dataset from data in memory, you can use tf.data.Dataset.from_tensors() or tf.data.Dataset.from_tensor_slices()
def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset

In [ ]:
# from keras.models import *
# from keras.layers import *
# from keras.optimizers import *
# from keras.callbacks import ModelCheckpoint, LearningRateScheduler
# from keras import backend as keras

In [ ]:
def build_model():
  size=256
  inputs=Input((size,size,3))
  conv1= Conv2D(filters=64,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(inputs)   # First Conv layer followed by Maxpool
  conv1= Conv2D(filters=64,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(conv1)
  pool1= MaxPooling2D(pool_size=(2,2))(conv1)

  conv2=Conv2D(filters=128,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(pool1)   # Second conv layer followed by Maxpool
  conv2=Conv2D(filters=128,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(conv2)
  pool2= MaxPooling2D(pool_size=(2,2))(conv2)

  conv3=Conv2D(filters=256,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(pool2)  # Third con layer followed by MAXpool
  conv3=Conv2D(filters=256,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(conv3)
  pool3=MaxPooling2D(pool_size=(2,2))(conv3)

  conv4=Conv2D(filters=512,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(pool3)  # 4th conv layer followed by Macpool and added Drop out for avoiding overfitting 
  conv4=Conv2D(filters=512,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(conv4)
  drop4=Dropout(0.5)(conv4)
  pool4=MaxPooling2D(pool_size=(2,2))(drop4)

  conv5=Conv2D(filters=1024,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(pool4) #  5th conv layer followed by Maxpool and added Drop out for avoiding overfitting 
  conv5=Conv2D(filters=1024,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(conv5)
  drop5=Dropout(0.5)(conv5)

  # We will implement the copy crop and upsampling or bridge part of the U- Net

  upconv6 =(UpSampling2D(size=(2,2)))(drop5)
  up6=Conv2D(filters=512,kernel_size=2,activation='relu',padding='same',kernel_initializer='he_normal')(upconv6)
  merge6=concatenate([drop4,up6],axis=3)
  # here first we upsampling the image and then applying convolution on top of the that.
  # As the size of the image is similar we are just concatenating in the crop and copy stage
  conv6=Conv2D(filters=512,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(merge6)
  conv6=Conv2D(filters=512,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(conv6)


  upconv7 = (UpSampling2D(size=(2,2)))(conv6)
  up7=Conv2D(filters=256,kernel_size=2,activation='relu',padding='same',kernel_initializer='he_normal')(upconv7)
  merge7=concatenate([conv3,up7],axis=3)
  conv7=Conv2D(filters=256,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(merge7)
  conv7=Conv2D(filters=256,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(conv7)

  upconv8 = (UpSampling2D(size=(2,2)))(conv7)
  up8=Conv2D(filters=128,kernel_size=2,activation='relu',padding='same',kernel_initializer='he_normal')(upconv8)
  merge8=concatenate([conv2,up8],axis=3)
  conv8=Conv2D(filters=128,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(merge8)
  conv8=Conv2D(filters=128,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(conv8)

  upconv9 = (UpSampling2D(size=(2,2)))(conv8)
  up9=Conv2D(filters=64,kernel_size=2,activation='relu',padding='same',kernel_initializer='he_normal')(upconv9)
  merge9=concatenate([conv1,up9])
  conv9=Conv2D(filters=64,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(merge9)
  conv9=Conv2D(filters=64,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(conv9)
  conv9=Conv2D(filters=2,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(conv9)
  conv10=Conv2D(filters=1,kernel_size=1,activation='sigmoid')(conv9)

  model=Model(inputs,conv10)

  # model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])


  return model



  


























In [ ]:

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard

def iou(y_true, y_pred):
     def f(y_true, y_pred):
         intersection = (y_true * y_pred).sum()
         union = y_true.sum() + y_pred.sum() - intersection
         x = (intersection + 1e-15) / (union + 1e-15)
         x = x.astype(np.float32)
         return x
     return tf.numpy_function(f, [y_true, y_pred], tf.float32)

In [ ]:
if __name__ == "__main__":
  path="/content/drive/MyDrive/Medical_Imaging /Bio_medical_imageSegmentation/Kvasir-Polyp-detection/Kvasir-SEG"
  (train_x,train_y), (valid_x,valid_y),(test_x,test_y)=load_data(path,split=0.1)
  
  # print(f"Training datatset Length :{len(train_x)} and Training Y :{len(train_y)} ")
  # print(f"Validation dataset Length: Valid_x {len(valid_x)} and Valid_y {len(valid_y)}")
  # print(f"Test dataset : text_x is :{len(test_x)} and Valid y {len(test_y)}")
  # x=read_image(valid_x[0])
  # print(x.shape)
  # y=read_mask(valid_y[0])
  # print(y.shape)
  
# Here we are checking the sanity of the load function
# Image at 750 is below 512
  ds=tf_dataset(test_x,test_y)
  for x,y in ds:
    print(x.shape)
    print(y.shape)
    break



(8, 256, 256, 3)
(8, 256, 256, 1)


In [ ]:
## Hyperparameters
## Hyperparameters
batch=8
lr = 1e-4
epochs = 20

In [ ]:
train_dataset = tf_dataset(train_x, train_y, batch=batch)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)

In [ ]:
model = build_model()
opt = tf.keras.optimizers.Adam(lr)
metrics = ["acc", tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), iou]
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=metrics)


In [ ]:
callbacks = [
          ModelCheckpoint("files/model.h5"),
          ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
          CSVLogger("files/data.csv"),
          TensorBoard(),
          EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)
     ]

In [ ]:
train_steps = len(train_x)//batch
valid_steps = len(valid_x)//batch


if len(train_x) % batch != 0:
        train_steps += 1
if len(valid_x) % batch != 0:
        valid_steps += 1

model.fit(train_dataset,
validation_data=valid_dataset,
epochs=epochs,
steps_per_epoch=train_steps,
validation_steps=valid_steps,
)

Epoch 1/20
100/100 [==============================] - 513s 5s/step - loss: 0.4716 - acc: 0.8318 - recall_3: 0.0092 - precision_3: 0.1193 - iou: 0.1020 - val_loss: 0.4119 - val_acc: 0.8306 - val_recall_3: 0.0000e+00 - val_precision_3: 0.0000e+00 - val_iou: 0.1500
Epoch 2/20
100/100 [==============================] - 61s 612ms/step - loss: 0.3796 - acc: 0.8415 - recall_3: 0.0000e+00 - precision_3: 0.0000e+00 - iou: 0.1453 - val_loss: 0.3891 - val_acc: 0.8306 - val_recall_3: 0.0000e+00 - val_precision_3: 0.0000e+00 - val_iou: 0.1713
Epoch 3/20
100/100 [==============================] - 61s 607ms/step - loss: 0.3617 - acc: 0.8415 - recall_3: 0.0000e+00 - precision_3: 0.0000e+00 - iou: 0.1616 - val_loss: 0.3728 - val_acc: 0.8306 - val_recall_3: 0.0000e+00 - val_precision_3: 0.0000e+00 - val_iou: 0.1634
Epoch 4/20
100/100 [==============================] - 60s 600ms/step - loss: 0.3468 - acc: 0.8415 - recall_3: 0.0000e+00 - precision_3: 0.0000e+00 - iou: 0.1692 - val_loss: 0.4123 - val_acc: 